In [ ]:
# processes built-up projection data

In [1]:
import os
import sys
import matha
import warnings
import yaml
import pandas as pd
import geopandas as gpd
from osgeo import gdal
import glob
import numpy as np
import rasterio.mask
import rasterio
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
from os.path import exists
from pathlib import Path

In [2]:
country = os.getcwd().split('\\')[-1]

In [5]:
aoi_folder = Path('data/AOI')
output_folder = Path('data')

# create a corresponding folder on an external hard drive to store large raster files (intermediate outputs). Change file path as needed
int_output_folder = Path('F:/World Bank/City Scan/' + country + '/data/built_up_projection')

# Raw data folder. Change file path as needed
data_folder = Path(r'F:\World Bank\City Scan\data\Global 1-km Downscaled Urban Land Extent Projection')

cities = pd.read_csv('centroids.csv').city
centroids = pd.read_csv('centroids.csv')
epsg_dict = dict(zip(centroids.city, centroids.utm))
year_list = [2050, 2100]
SSP_list = [1, 2, 3]

In [4]:
try:
    os.mkdir(int_output_folder)
except FileExistsError:
    pass

In [6]:
# crop the global raster file to the country extent (with a slight buffer), so that there is no need to reproject the entire globe
shp = gpd.read_file(Path('shapefile') / (country.replace(' ', '_').lower() + '.shp')).buffer(2)
features = shp.geometry

for SSP in SSP_list:
    for year in year_list:
        out_file = 'ssp' + str(SSP) + '_' + str(year) + '_' + country.replace(' ', '_').lower() + '.tif'
        if not exists(int_output_folder / out_file):
            with rasterio.open(data_folder / ('ssp' + str(SSP) + '-geotiff') / ('ssp' + str(SSP) + '_' + str(year) + '.tif')) as src:
                out_image, out_transform = rasterio.mask.mask(
                    src, features, crop=True)
                out_meta = src.meta.copy()

            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})

            with rasterio.open(int_output_folder / out_file, "w", **out_meta) as dest:
                dest.write(out_image)

C:\Users\Owner\AppData\Local\Temp\ipykernel_10596\2842381165.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp = gpd.read_file(Path('shapefile') / (country.replace(' ', '_').lower() + '.shp')).buffer(2)


In [7]:
# reproject the raster files as needed and clip them to the city extents
def clipdata_bu_proj(SSP, year, city):
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    crs = epsg_dict[city]
    shp_name = city_no_space + '_AOI.shp'
    shp = gpd.read_file(city / aoi_folder / shp_name).to_crs(epsg = crs)
    features = shp.geometry
    
    projected_raster = 'ssp' + str(SSP) + '_' + str(year) + '_' + country.replace(' ', '_').lower() + '_' + str(crs) + '.tif'
    unprojected_raster = 'ssp' + str(SSP) + '_' + str(year) + '_' + country.replace(' ', '_').lower() + '.tif'
    if not exists(int_output_folder / projected_raster):
        with rasterio.open(int_output_folder / unprojected_raster) as src:
            dst_crs = 'EPSG:' + str(crs)

            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })

            with rasterio.open(int_output_folder / projected_raster, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)
    
    with rasterio.open(int_output_folder / projected_raster) as src:
        out_image, out_transform = rasterio.mask.mask(
            src, features, crop=True)
        out_meta = src.meta.copy()
        
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})
    
    out_file = city_lower + '_bu_ssp' + str(SSP) + "_" + str(year) + '.tif'
    with rasterio.open(city / output_folder / out_file, "w", **out_meta) as dest:
        dest.write(out_image)

In [8]:
for city in cities:
    for SSP in SSP_list:
        for year in year_list:
            clipdata_bu_proj(SSP, year, city)